In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import re
import time
import numpy as np
import pandas as pd
from get_sankey_data import get_moviedata 
import sqlalchemy
from sqlalchemy import create_engine

In [49]:
#2018top100
#url=("https://www.boxofficemojo.com/yearly/chart/?yr=2018&p=.htm")
#2017top100
url=("https://www.boxofficemojo.com/yearly/chart/?yr=2017&p=.htm")

movie_links=[]

r = requests.get(url)
soup = BeautifulSoup(r.text,"lxml")
links = soup.find_all("a", href = re.compile("id="))
for link in links:
    l = "https://www.boxofficemojo.com/movies/?page=intl&" + link["href"].split("?")[1]
    movie_links.append(l)
movie_links.pop(0)
movie_links = movie_links[0:10]

In [31]:
#mcu
#url=("https://www.boxofficemojo.com/franchises/chart/?id=avengers.htm")
#StarWars
#url=("https://www.boxofficemojo.com/franchises/chart/?id=starwars.htm")
#JKRolling
#url=("https://www.boxofficemojo.com/franchises/chart/?id=jkrowling.htm")


movie_links=[]

r = requests.get(url)
soup = BeautifulSoup(r.text,"lxml")
links = soup.find_all("a", href = re.compile("id="))
for link in links:
    if link.b:
        l = "https://www.boxofficemojo.com/movies/?page=intl&" + link["href"].split("?")[1]
        if l not in movie_links:
            movie_links.append(l)
movie_links.pop(0)

'https://www.boxofficemojo.com/movies/?page=intl&view=main&id=jkrowling.htm&sort=gross&order=ASC&p=.htm'

In [50]:
movie_links

['https://www.boxofficemojo.com/movies/?page=intl&id=starwars8.htm',
 'https://www.boxofficemojo.com/movies/?page=intl&id=beautyandthebeast2017.htm',
 'https://www.boxofficemojo.com/movies/?page=intl&id=wonderwoman.htm',
 'https://www.boxofficemojo.com/movies/?page=intl&id=jumanji2016.htm',
 'https://www.boxofficemojo.com/movies/?page=intl&id=marvel17a.htm',
 'https://www.boxofficemojo.com/movies/?page=intl&id=spiderman2017.htm',
 'https://www.boxofficemojo.com/movies/?page=intl&id=it.htm',
 'https://www.boxofficemojo.com/movies/?page=intl&id=marvel2017.htm',
 'https://www.boxofficemojo.com/movies/?page=intl&id=despicableme3.htm',
 'https://www.boxofficemojo.com/movies/?page=intl&id=dcfilm1117.htm']

In [51]:
concat_table = get_moviedata(movie_links)

/Users/YoYoYe/Desktop/RutgersDataScienceBootcamp/Project/movie_visualizations/Youqing/data scraping/get_sankey_data.py:118: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  concat_table = pd.concat([concat_table,final_table])


In [52]:
concat_table.source = concat_table.source.replace("Harry Potter and the ","",regex=True)
concat_table.target = concat_table.target.replace("Harry Potter and the ","",regex=True)
concat_table.head()

,source,target,value
0,USA,Star Wars:The Last Jedi Domestic,620181382
1,Star Wars:The Last Jedi Domestic,Star Wars:The Last Jedi,620181382
2,Star Wars:The Last Jedi Foreign,Star Wars:The Last Jedi,712358507
3,United Kingdom,Star Wars:The Last Jedi Foreign,111106152
4,Germany,Star Wars:The Last Jedi Foreign,83164496


In [53]:
engine = create_engine("sqlite:///../foreign_gross/db/movies.sqlite", echo=False)

In [54]:
concat_table.to_sql("seventeen", con=engine,if_exists="replace")

In [ ]:
engine.execute("SELECT * FROM jkr").fetchall()

In [38]:
concat_table.to_csv("../foreign_gross/static/data/jkr.csv",encoding="utf-8")    